In [14]:
import pandas as pd
import numpy as np
import ast
import pickle

In [ ]:
#pip install wikipedia-api

In [12]:
import wikipediaapi


wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

#  Get data for each category

In [3]:
from random import shuffle


def url_categorymembers(categorymembers,list_pages=[], level=0, max_level=3, i=0):
    pages_cat = list(categorymembers.values()) #toutes les pages de la catégorie
    shuffle(pages_cat)

    for c in pages_cat:
        if i==1000: #1000 pages par catégorie
            break

        if c.title[:6]!="Portal" and c.title[:5]!="File:" and c.title[:8]!="Category": # vérifie que c'est bien un article
            i+=1
            d = {}
            d["titre"] = c.title
            d["contenu"] = c.text
            d["liens"] = list(c.links.keys())
            list_pages.append(d)

        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level: # descend d'un niveau
            i = url_categorymembers(c.categorymembers, list_pages, level=level + 1, max_level=max_level, i=i)[1]

    return list_pages, i

In [ ]:
cat_arts = wiki_wiki.page("Category:The arts") # Arts
arts, _ = url_categorymembers(cat_arts.categorymembers)
pd.DataFrame(arts).to_csv('arts.csv')

In [ ]:
cat_games = wiki_wiki.page("Category:Games") # Games
games, _= url_categorymembers(cat_games.categorymembers)

pd.DataFrame(games).to_csv('games.csv')

In [ ]:
cat_youth = wiki_wiki.page("Category:Youth") # Kids and Teens (not exact)
youth, _ = url_categorymembers(cat_youth.categorymembers)

pd.DataFrame(youth).to_csv('youth.csv')

In [ ]:
cat_reference = wiki_wiki.page("Category:Reference") # Reference
reference, _ = url_categorymembers(cat_reference.categorymembers)

pd.DataFrame(reference).to_csv('reference.csv')

In [ ]:
cat_shopping = wiki_wiki.page("Category:Retailing") # Shopping
shopping, _ = url_categorymembers(cat_shopping.categorymembers)

pd.DataFrame(shopping).to_csv('shopping.csv')

In [ ]:
cat_business = wiki_wiki.page("Category:Business") # Business
business, _ = url_categorymembers(cat_business.categorymembers)

pd.DataFrame(business).to_csv('business.csv')

In [ ]:
cat_health = wiki_wiki.page("Category:Health") # Health
health, _ = url_categorymembers(cat_health.categorymembers)

pd.DataFrame(health).to_csv('health.csv')

In [ ]:
cat_news = wiki_wiki.page("Category:News") # News
news, _ = url_categorymembers(cat_news.categorymembers)

pd.DataFrame(news).to_csv('news.csv')

In [ ]:
cat_geography = wiki_wiki.page("Category:Geography") # Regional (not exact)
geography, _ = url_categorymembers(cat_geography.categorymembers)

pd.DataFrame(geography).to_csv('geography.csv')

In [ ]:
cat_society = wiki_wiki.page("Category:Society") # Society
society, _ = url_categorymembers(cat_society.categorymembers)

pd.DataFrame(society).to_csv('society.csv')

In [ ]:
cat_computers = wiki_wiki.page("Category:Computers") # Computers
computers, _ = url_categorymembers(cat_computers.categorymembers)

pd.DataFrame(computers).to_csv('computers.csv')

In [ ]:
cat_home = wiki_wiki.page("Category:Home") # Home
home, _ = url_categorymembers(cat_home.categorymembers)

pd.DataFrame(home).to_csv('home.csv')

In [ ]:
cat_recreation = wiki_wiki.page("Category:Recreation") # Recreation
recreation, _ = url_categorymembers(cat_recreation.categorymembers)

pd.DataFrame(recreation).to_csv('recreation.csv')

In [ ]:
cat_science = wiki_wiki.page("Category:Science") # Science
science, _ = url_categorymembers(cat_science.categorymembers)

pd.DataFrame(science).to_csv('science.csv')

In [ ]:
cat_sports = wiki_wiki.page("Category:Sports") # Sports
sports, _ = url_categorymembers(cat_sports.categorymembers)

pd.DataFrame(sports).to_csv('sports.csv')

In [ ]:
cat_world = wiki_wiki.page("Category:World") # World
world, _ = url_categorymembers(cat_world.categorymembers)

pd.DataFrame(world).to_csv('world.csv')

In [53]:
csv_files = [
    'arts.csv', 'business.csv', 'computers.csv', 'games.csv',
    'geography.csv', 'health.csv', 'home.csv', 'news.csv',
    'recreation.csv', 'reference.csv', 'science.csv', 'shopping.csv',
    'society.csv', 'sports.csv', 'world.csv', 'youth.csv'
]

# Liste pour stocker les DataFrames
dataframes = []

# Lire chaque fichier CSV et l'ajouter à la liste des DataFrames
for file in csv_files:
    df = pd.read_csv(f'./Data/{file}')
    dataframes.append(df)

# Combiner tous les DataFrames en un seul
combined_df = pd.concat(dataframes, ignore_index=True)

# Écrire le DataFrame combiné dans un nouveau fichier CSV
combined_df.to_csv('./Data/all.csv', index=False)

# Compute Page Rank vectors (3.2)

## Matrices d'adjacence

Construction de la matrice sans utiliser les fichiers :

In [4]:
# build the adjacency matrix
def build_adjacency_matrix_2(df): #pas optimisé
    num_pages = df.shape[0]
    adjacency_matrix = np.zeros((num_pages, num_pages))
    df["liens"] = df["liens"].apply(ast.literal_eval)
    
    for index, row in df.iterrows():
        N = len(np.where(np.isin(row['liens'],df['titre'].values))[0]) # nombres de liens que fait la page
        for link in row['liens']:
            if link in df['titre'].values:
                j = np.where(df['titre'] == link )[0]
                adjacency_matrix[j,index] += 1/N

    return adjacency_matrix



def build_adjacency_matrix(df):
    df["liens"] = df["liens"].apply(ast.literal_eval)  # Convertir les liens de type string en listes

    # Créer un dictionnaire pour mapper les titres aux indices
    title_to_index = {title: i for i, title in enumerate(df['titre'])}
    
    # Initialiser la matrice d'adjacence avec des zéros
    num_pages = len(df)
    adjacency_matrix = np.zeros((num_pages, num_pages))
    
    # Construire une série pour mapper les liens aux indices
    link_series = pd.Series(df['liens'].values.tolist())
    link_indices = link_series.map(lambda x: [title_to_index[link] for link in x if link in title_to_index])
    
    # Compter le nombre de liens sortants pour chaque article
    df['num_liens'] = link_indices.apply(len)
    
    for i, indices in enumerate(link_indices):
        if len(indices)>0:
            adjacency_matrix[indices, i] = 1 / df.at[i, 'num_liens']
    
    return adjacency_matrix

In [108]:
matrice_all= build_adjacency_matrix(pd.read_csv('./Data/all.csv'))
np.save('./Matrices/matrice_all.npy', matrice_all)

Construction de la matrice en utilisant les fichiers :

In [ ]:
fichier_graphe = pd.read_csv('enwiki.wikilink_graph.2018-03-01.csv')

# build the adjacency matrix
def build_adjacency_matrix(topic_articles, inversed_index_articles):
    num_pages = len(topic_articles)
    adjacency_matrix = np.zeros((num_pages, num_pages))

    for i,titre in enumerate(topic_articles):
        #links = page_links(pages[i])
        links = fichier_graphe[fichier_graphe['page_title_from']==titre]
        for link in links:
            if link in topic_articles:
                j = topic_articles.index(link)
                adjacency_matrix[j][i] = 1

    return adjacency_matrix/sum(adjacency_matrix,axis=0)



## Calcul des vecteurs

In [13]:
# compute PageRank scores
def compute_pagerank(M, alpha=0.25, bias = None): # bias = None pour page_rank normal, bias = topic sinon
    # M = adjacency_matrix
    # damping_factor = 1-alpha

    N = M.shape[0]

    if bias is None : # page_rank normal, pas topic_sensitive
        p = np.ones(N) / N 

    else: # ODP-biasing 
        articles = pd.read_csv(f'./Data/all.csv')["titre"].values
        articles_cat = pd.read_csv(f'./Data/{bias}.csv')["titre"].values
        p = np.where(np.isin(articles,articles_cat), 1/len(articles_cat), 0)

    #M_prime = (1-alpha) * M + alpha * teleportation_matrix

    rank = np.ones(N) / N
    old_rank = np.zeros(N)

    epsilon = 1.0e-3
    max_iterations = 10
    iterations = 0

    while np.sum(np.abs(rank - old_rank)) > epsilon and iterations < max_iterations:
        old_rank = rank.copy()
        rank = (1-alpha)*M @ rank + p # equation 5
        #normaliser ?
        iterations += 1

    return rank

In [2]:
# Compute PageRank scores for each topic
import numpy as np

Topic_names = ["arts","games","youth","reference", "shopping", "business", "health", "news","geography","society","computers","home","recreation","science","sports","world"]

adjacency_matrix = np.load(f'./Matrices/matrice_all.npy')

topic_pagerank = {}
for topic in Topic_names:
    #adjacency_matrix = np.load(f'./Matrices/matrice_{topic}.npy')
    
    pagerank_scores = compute_pagerank(adjacency_matrix,bias=topic)

    print(f"Topic: {topic}")
    print(pagerank_scores[:5])

    topic_pagerank[topic] = pagerank_scores

    

FileNotFoundError: [Errno 2] No such file or directory: './Matrices/matrice_all.npy'

In [114]:
with open('topic_pagerank.pkl', 'wb') as fichier:
    pickle.dump(topic_pagerank, fichier)

In [ ]:
adjacency_matrix = np.load(f'./Matrices/matrice_all.npy')
pagerank_normal = compute_pagerank(adjacency_matrix,bias=None)
with open('normal_pagerank.pkl', 'wb') as fichier:
    pickle.dump(pagerank_normal, fichier)

# Query-time importance score (3.3)

## Calcul de D

In [1]:
import json
from sklearn.feature_extraction.text import CountVectorizer


categories_text = {}
for c in Topic_names:
    liste_textes = pd.read_csv(f'./Data/{c}.csv')["contenu"].values.astype(str)

    texte = ' '.join(liste_textes)
    categories_text[c] = texte

D_categories = {}
terms_indexes_categories = {}

for c, text in categories_text.items():
    vectorizer = CountVectorizer(stop_words='english')
    D_categories[c] = vectorizer.fit_transform([text])
    terms_indexes_categories[c] = vectorizer.get_feature_names_out()
    

NameError: name 'Topic_names' is not defined

In [ ]:
with open('D.pkl', 'wb') as fichier:
    pickle.dump(D_categories, fichier)

with open('terms_indexes.pkl', 'wb') as fichier:
    pickle.dump(terms_indexes_categories, fichier)

## Score

In [ ]:
!pip install python-terrier

In [3]:
# Indexation des articles 
import pandas as pd
import pyterrier as pt

if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

df = pd.read_csv(f'./Data/all.csv')[["titre","contenu"]]
df = df.dropna()
df.columns = ["docno", "text"]

# pd_indexer = pt.DFIndexer("./pd_index")
# indexref = pd_indexer.index(df["text"], df["docno"])
# index = pt.IndexFactory.of(indexref)

Topic_names = ["arts","games","youth","reference", "shopping", "business", "health", "news","geography","society","computers","home","recreation","science","sports","world"]

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
df = pd.read_csv(f'./Data/all.csv')[["titre","contenu"]]
df = df.fillna('missing')

In [36]:
def get_probas_c_q(q, D, terms_indexes):
    # calculer P(cj|q)
    probas_c_q = np.zeros((16))
    p_c = 1/16 # "The quantity P(cj) is not as straightforward. We chose to make it uniform"

    for i,topic in enumerate(Topic_names):
        indexes = np.in1d(terms_indexes[topic], np.array(q)).nonzero()[0]

        if len(indexes)==0:
            probas_c_q[i] = 0
        else:
            p_q_c = D[topic].tocsc()[indexes]/D[topic].tocsc().sum()
            p_q_c = p_q_c.toarray()
            probas_c_q[i] = p_c * np.prod(p_q_c)
            
            
    if np.sum(probas_c_q)==0:
        probas_c_q = np.ones(16)/16
    else:
        probas_c_q = probas_c_q/np.sum(probas_c_q)
            
    return probas_c_q


def sqd(q, topic_pagerank, D, terms_indexes, index):
    # Using a text index, recuperer les docs qui contiennent q
    br = pt.BatchRetrieve(index, wmodel="CoordinateMatch") # CoordinateMatch renvoie 1 si le terme est dans le doc, 0 sinon
    docs_retrieved = br.search(q) # Renvoie le nombre de termes de la query qui sont dans chaque doc
    docs_query = docs_retrieved[docs_retrieved["score"] == len(q.split())]  # Garder uniquement les docs qui contiennent tous les termes de la query
    docs_index = docs_query["docno"].values
    
    probas_c_q = get_probas_c_q(q, D, terms_indexes)
            
    df = pd.read_csv(f'./Data/all.csv')
    df.columns = ["id", "titre", "contenu", "liens"]
            
    docs = df[df["titre"].isin(docs_index)].copy()
    
    # print(docs)
    
    docs["score"] = docs.apply(lambda x: sum([probas_c_q[Topic_names.index(topic)]*topic_pagerank[topic][x["id"]] for topic in Topic_names]), axis=1)
    
    # print(docs)

    # garder les 3 meilleurs cj
    # sorted_indices = np.argsort(-probas_c_q)
    # 
    # top_indices = sorted_indices[:3]
    # top_values = probas_c_q[top_indices]
    # 
    # # somme des P(cj|q) * rank j
    # res = 0
    # for i in range(3):
    #     res+= top_values[i]*topic_pagerank[Topic_names[top_indices[i]]]
    #return np.sum(top_values*topic_pagerank[top_indices],axis=1)


    # meilleurs documents :

    # doc qui ont les mots de la query sans pyterrier
    # df['text_lower'] = df['contenu'].str.lower() # Convert the text column to lowercase for case-insensitive search
    # query_words = q.lower().split() # Split the query into individual words
    # mask = df['text_lower'].str.contains('|'.join(query_words)) # boolean mask to filter rows containing any of the query words
    # doc_indices = df.index[mask] # indices des docs qui ont la query
    
    #res[doc_indices]+=100 # on leur donne plus de valeur pour qu'ils sortent d'abord

    #sorted_docs_index = np.argsort(-res)
    # best_docs = pd.read_csv(f'./Data/all.csv')["titre"].values[sorted_docs_index]

    best_docs = docs.sort_values(by="score", ascending=False)["titre"]
    return best_docs

In [6]:
# query = "affirmative action"
# print(sqd(query, topic_pagerank, D, terms_indexes, index)[:5])

In [7]:
def score_normal_pagerank(q,normal_pagerank):
    
    # # doc qui ont les mots de la query (sans pyterrier)
    # df['text_lower'] = df['contenu'].str.lower() # Convert the text column to lowercase for case-insensitive search
    # query_words = q.lower().split() # Split the query into individual words
    # mask = df['text_lower'].str.contains('|'.join(query_words)) # boolean mask to filter rows containing any of the query words
    # doc_indices = df.index[mask] # indices des docs qui ont la query
    # normal_pagerank[doc_indices] +=100
    # 
    # sorted_docs_index = np.argsort(-normal_pagerank)
    # best_docs = pd.read_csv(f'./Data/all.csv')["titre"].values[sorted_docs_index]
    
    # Using a text index, recuperer les docs qui contiennent q
    br = pt.BatchRetrieve(index, wmodel="CoordinateMatch") # CoordinateMatch renvoie 1 si le terme est dans le doc, 0 sinon
    docs_retrieved = br.search(q) # Renvoie le nombre de termes de la query qui sont dans chaque doc
    docs_query = docs_retrieved[docs_retrieved["score"] == len(q.split())]  # Garder uniquement les docs qui contiennent tous les termes de la query
    sorted_index = docs_query["docno"].values
    
    df = pd.read_csv(f'./Data/all.csv')
    df.columns = ["id", "titre", "contenu", "liens"]
    
    docs = df[df["titre"].isin(sorted_index)].copy()
    docs["score"] = normal_pagerank[docs["id"].values]
    
    best_docs = docs.sort_values(by="score", ascending=False)["titre"]
    
    return best_docs


# Experimental results

## Similarity ranking (4.1)

In [8]:
def OSim(t1, t2, n=20):
    # t1, t2 listes
    return len(set(t1[:n])&set(t2[:n]))/n

In [9]:
import itertools

def KSim(t1, t2):
    # t1, t2 listes
    U = set(t1)|set(t2)
    delta1 = U - set(t1)
    delta2 = U - set(t2)

    t1_prime = list(t1)+list(delta1)
    t2_prime = list(t2)+list(delta2)

    sim = 0
    for u,v in list(itertools.permutations(U, 2)):
        if np.sign(t1_prime.index(u)-t1_prime.index(v))==np.sign(t2_prime.index(u)-t2_prime.index(v)):
            sim+=1
    return sim/(len(U)*(len(U)-1)) if len(U)>1 else sim


## Tests on queries

In [10]:
queries = [ "affirmative action", "alcoholism", "amusement parks", 
            "architecture", "bicycling", "blues", "cheese", 
            "citrus groves", "classical guitar", "computer vision", 
            "cruises", "death valley", "field hockey", 
            "gardening", "graphic design", "gulf war", 
            "hiv", "java", "lipari",
            "lyme disease", "mutual funds", "national parks", 
            "parallel architecture", "recycling cans", "rock climbing", 
            "san francisco", "shakespeare", "stamp collecting", 
            "sushi", "table tennis", "telecommuting", 
            "vintage cars", "volcano", "zen buddhism", "zener"]

In [11]:
import numpy as np
import pickle

with open("terms_indexes.pkl", "rb") as f:
    terms_indexes = pickle.load(f)
with open("D.pkl", "rb") as f:
    D = pickle.load(f)
    for topic in Topic_names:
        D[topic] = np.reshape(D[topic],(D[topic].shape[1],1))
with open("topic_pagerank.pkl", "rb") as f:
    topic_pagerank = pickle.load(f)
    
index = pt.IndexFactory.of("./pd_index")

In [12]:
for q in queries[:5]:
    print(q)
    print(sqd(q, topic_pagerank, D, terms_indexes, index)[:5])


affirmative action
8        Expressive therapies
11499        Apple and unions
12423           Shadow family
12384           Public sphere
12357    Gender mainstreaming
Name: titre, dtype: object
alcoholism
15584           Shameless (British TV series)
13729                     I will moida da bum
8276                 Violence and video games
5827                      Million Women Study
10992    List of Stargate Universe characters
Name: titre, dtype: object
amusement parks
85         Dream world (plot device)
8879                            Park
8762                         Soarin'
8758    Lists of tourist attractions
8743              Tourist attraction
Name: titre, dtype: object
architecture
750                          Art
8750                      Museum
10584      Mapping controversies
12584    Heritage interpretation
5584         Imaging informatics
Name: titre, dtype: object
bicycling
6273                                        Pannier
758                                  Conc

In [13]:
with open("normal_pagerank.pkl", "rb") as f:
        normal_pagerank = pickle.load(f)

In [42]:
for q in queries[:5]:
    print(q)
    print(score_normal_pagerank(q, normal_pagerank)[:5])

affirmative action
14017    Universal Declaration of Human Rights
10992     List of Stargate Universe characters
7810                           Murray Rothbard
14841          Global Alliance for EcoMobility
7101                               Jactitation
Name: titre, dtype: object
alcoholism
15584    Shameless (British TV series)
13729              I will moida da bum
5827               Million Women Study
8276          Violence and video games
6156                      Chafing dish
Name: titre, dtype: object
amusement parks
8584                            Hobby
8758     Lists of tourist attractions
14148                       Ice cream
7748              Japanese war crimes
10994                 Kenny McCormick
Name: titre, dtype: object
architecture
750                          Art
8750                      Museum
12584    Heritage interpretation
10584      Mapping controversies
5584         Imaging informatics
Name: titre, dtype: object
bicycling
6273                                  

In [43]:
for q in queries[:5]:
    print(q)
    print(score_normal_pagerank(q, topic_pagerank['business'])[:5])

affirmative action
10992     List of Stargate Universe characters
14017    Universal Declaration of Human Rights
9991          Chronicle of the King D. Pedro I
7101                               Jactitation
748                              Theory of art
Name: titre, dtype: object
alcoholism
6922                                   Skewer
12922                            Public enemy
11958                                  Sandoz
8969                               Maxim Wien
10992    List of Stargate Universe characters
Name: titre, dtype: object
amusement parks
10958                          Tom Swift
8136               Video games and Linux
3996     Characters of the Tekken series
8758        Lists of tourist attractions
10994                    Kenny McCormick
Name: titre, dtype: object
architecture
8962                                 Reserve design
969                                          Design
272                                       Patronage
3272    List of Xbox games compati

In [44]:
for q in queries[:5]:
    print(q)
    print(score_normal_pagerank(q, topic_pagerank['sports'])[:5])

affirmative action
14017    Universal Declaration of Human Rights
7810                           Murray Rothbard
7101                               Jactitation
10992     List of Stargate Universe characters
14841          Global Alliance for EcoMobility
Name: titre, dtype: object
alcoholism
5827              Million Women Study
5934     Heart failure classification
8969                       Maxim Wien
11957                           Maggi
5926                   Medical record
Name: titre, dtype: object
amusement parks
8758     Lists of tourist attractions
8584                            Hobby
10958                       Tom Swift
14148                       Ice cream
10994                 Kenny McCormick
Name: titre, dtype: object
architecture
750                                              Art
8750                                          Museum
2827                                           Intel
8962                                  Reserve design
10934    List of things named aft

## Similarité entre les ranking de pagerank normal et de pagerank avec topic

In [38]:
for q in queries:
    normal = score_normal_pagerank(q, normal_pagerank)
    topic = sqd(q, topic_pagerank, D, terms_indexes, index)
    print(q,"\tOSim:",OSim(normal, topic, n=20))
    #print("KSim,",KSim(normal[:20], topic[:20]))

affirmative action 	OSim: 1.0
alcoholism 	OSim: 0.9
amusement parks 	OSim: 1.0
architecture 	OSim: 1.0
bicycling 	OSim: 0.9
blues 	OSim: 0.85
cheese 	OSim: 0.7
citrus groves 	OSim: 0.15
classical guitar 	OSim: 0.9
computer vision 	OSim: 1.0
cruises 	OSim: 0.75
death valley 	OSim: 0.95
field hockey 	OSim: 1.0
gardening 	OSim: 0.6
graphic design 	OSim: 1.0
gulf war 	OSim: 1.0
hiv 	OSim: 0.8
java 	OSim: 0.85
lipari 	OSim: 0.05
lyme disease 	OSim: 0.2
mutual funds 	OSim: 1.0
national parks 	OSim: 1.0
parallel architecture 	OSim: 0.9
recycling cans 	OSim: 0.95
rock climbing 	OSim: 0.95
san francisco 	OSim: 1.0
shakespeare 	OSim: 0.95
stamp collecting 	OSim: 0.95
sushi 	OSim: 0.85
table tennis 	OSim: 0.9
telecommuting 	OSim: 0.1
vintage cars 	OSim: 1.0
volcano 	OSim: 0.85
zen buddhism 	OSim: 0.5
zener 	OSim: 0.15


In [51]:
for q in queries:
    normal = score_normal_pagerank(q, normal_pagerank)
    topic = sqd(q, topic_pagerank, D, terms_indexes, index)
    print(q,"\tKSim,",KSim(normal[:20], topic[:20]))

affirmative action 	KSim, 0.2927927927927928
alcoholism 	KSim, 0.8484848484848485
amusement parks 	KSim, 0.3548387096774194
architecture 	KSim, 0.9526315789473684
bicycling 	KSim, 0.8831168831168831
blues 	KSim, 0.8023715415019763
cheese 	KSim, 0.7476923076923077
citrus groves 	KSim, 0.3333333333333333
classical guitar 	KSim, 0.45454545454545453
computer vision 	KSim, 0.24444444444444444
cruises 	KSim, 0.6633333333333333
death valley 	KSim, 0.2838709677419355
field hockey 	KSim, 0.3092436974789916
gardening 	KSim, 0.5079365079365079
graphic design 	KSim, 0.21923076923076923
gulf war 	KSim, 0.3314393939393939
hiv 	KSim, 0.8268398268398268
java 	KSim, 0.7707509881422925
lipari 	KSim, 0
lyme disease 	KSim, 0.6666666666666666
mutual funds 	KSim, 0.2702702702702703
national parks 	KSim, 0.23616734143049933
parallel architecture 	KSim, 0.41935483870967744
recycling cans 	KSim, 0.26282051282051283
rock climbing 	KSim, 0.42610837438423643
san francisco 	KSim, 0.23076923076923078
shakespeare 	K

### Comparaison des vecteurs PageRank (table 4)

In [16]:
pageranks = topic_pagerank.copy()
pageranks["normal"] = normal_pagerank

In [39]:
import json

table4={}

# for topic in Topic_names + ["normal"]:
#     with open(f"Rankings/{topic}.json", 'w') as f:
#         rankings = {}
#         for q in queries:
#             rankings[q] = score_normal_pagerank(q, pageranks[topic])[:20].tolist()
#         json.dump(rankings, f, indent=4)

for topic_1,topic_2 in list(itertools.permutations(["normal"]+Topic_names, 2)):
    print(topic_1,topic_2)
    ksim = 0
    osim = 0
    
    with open(f"Rankings/{topic_1}.json", 'r') as f:
        rankings1 = json.load(f)
    with open(f"Rankings/{topic_2}.json", 'r') as f:
        rankings2 = json.load(f)
    
    for q in queries:
        indexation1 = rankings1[q]
        indexation2 = rankings2[q]
        ksim += KSim(indexation1, indexation2)
        osim += OSim(indexation1, indexation2)
    ksim /= len(queries)
    osim /= len(queries)
    table4[(topic_1, topic_2)] = (ksim, osim)

    

normal arts
normal games
normal youth
normal reference
normal shopping
normal business
normal health
normal news
normal geography
normal society
normal computers
normal home
normal recreation
normal science
normal sports
normal world
arts normal
arts games
arts youth
arts reference
arts shopping
arts business
arts health
arts news
arts geography
arts society
arts computers
arts home
arts recreation
arts science
arts sports
arts world
games normal
games arts
games youth
games reference
games shopping
games business
games health
games news
games geography
games society
games computers
games home
games recreation
games science
games sports
games world
youth normal
youth arts
youth games
youth reference
youth shopping
youth business
youth health
youth news
youth geography
youth society
youth computers
youth home
youth recreation
youth science
youth sports
youth world
reference normal
reference arts
reference games
reference youth
reference shopping
reference business
reference health
refer

In [53]:
l = sorted(table4.items(), key=lambda x: sum(x[1]), reverse=True)

for i in l[::2]:
    print(f"{i[0][0]} - {i[0][1]} : Ksim={i[1][0]}, Osim={i[1][1]}")

normal - arts : Ksim=0.790932208445121, Osim=0.7042857142857143
shopping - business : Ksim=0.6970292467498356, Osim=0.6885714285714284
health - computers : Ksim=0.6899202925603469, Osim=0.6871428571428572
sports - shopping : Ksim=0.6976597088789994, Osim=0.6728571428571429
normal - science : Ksim=0.7028570315198859, Osim=0.6671428571428569
sports - business : Ksim=0.6907902392487091, Osim=0.6642857142857145
society - world : Ksim=0.690537120299145, Osim=0.6614285714285715
games - recreation : Ksim=0.6901786374135511, Osim=0.66
business - computers : Ksim=0.6787755369342321, Osim=0.6671428571428569
sports - computers : Ksim=0.6788279844229664, Osim=0.657142857142857
business - health : Ksim=0.6709056394418256, Osim=0.6585714285714286
shopping - computers : Ksim=0.6749923879109713, Osim=0.65
world - youth : Ksim=0.6595662667692755, Osim=0.6599999999999998
normal - computers : Ksim=0.6716603599575106, Osim=0.6457142857142857
normal - geography : Ksim=0.6876609907432101, Osim=0.62857142857

In [55]:
t = ("games", "world")

a = [i for i in l if i[0][0] == t[0] and i[0][1] == t[1]]

for i in a:
    print(f"{i[0][0]} - {i[0][1]} : Ksim={i[1][0]}, Osim={i[1][1]}")

games - world : Ksim=0.5642471118022193, Osim=0.5399999999999999


## Table 6

In [37]:
for q in queries:
    probas = get_probas_c_q(q, D, terms_indexes)
    print(q)
    for i in np.argsort(-probas)[:3]:
        print(f"{Topic_names[i]}: {probas[i]}")
        
    print()

affirmative action
arts: 0.0625
games: 0.0625
youth: 0.0625

alcoholism
society: 0.31138485687527107
health: 0.19262831108363515
arts: 0.10568282429866296

amusement parks
arts: 0.0625
games: 0.0625
youth: 0.0625

architecture
computers: 0.42149124391537307
arts: 0.13929719604111984
health: 0.07928210669302674

bicycling
sports: 0.40679186780159465
world: 0.2482707271924775
recreation: 0.10682571994341371

blues
youth: 0.5887702960139773
arts: 0.17032078470585055
reference: 0.08722817897319757

cheese
world: 0.4694664697777181
geography: 0.27097612960093775
home: 0.12013278612529696

citrus groves
arts: 0.0625
games: 0.0625
youth: 0.0625

classical guitar
arts: 0.0625
games: 0.0625
youth: 0.0625

computer vision
arts: 0.0625
games: 0.0625
youth: 0.0625

cruises
home: 0.28526763631035906
reference: 0.16847105410131505
world: 0.1346733832289736

death valley
arts: 0.0625
games: 0.0625
youth: 0.0625

field hockey
arts: 0.0625
games: 0.0625
youth: 0.0625

gardening
society: 0.2477242956578

## Table 7

In [68]:
queries = ["alcoholism", "bicycling", "citrus groves", "computer vision", "death valley", "graphic design", "gulf war", "hiv", "shakespeare", "table tennis"]

for q in queries:
    print(q)
    
    with open(f"Rankings/normal.json", 'r') as f:
        normal = json.load(f)
        ranking_normal = normal[q]
        
    ranking_topic = sqd(q, topic_pagerank, D, terms_indexes, index).tolist()
    
    
    print("Normal PageRank")
    for i in range(min(10, len(ranking_normal))):
        print(f"{i+1}. {ranking_normal[i]}")
        
    print("\nTopic-sensitive PageRank")
    for i in range(min(10, len(ranking_topic))):
        print(f"{i+1}. {ranking_topic[i]}")
        
    print()




alcoholism
Normal PageRank
1. Shameless (British TV series)
2. I will moida da bum
3. Million Women Study
4. Violence and video games
5. Chafing dish
6. Maxim Wien
7. List of Stargate Universe characters
8. Drug control classes (UK)
9. Crowd abuse
10. Outline of wine

Topic-sensitive PageRank
1. Shameless (British TV series)
2. I will moida da bum
3. Violence and video games
4. Million Women Study
5. List of Stargate Universe characters
6. Maxim Wien
7. Chafing dish
8. Crowd abuse
9. Outline of wine
10. List of Fate/Grand Order characters

bicycling
Normal PageRank
1. Pannier
2. Conceptual art
3. Intel
4. Global Alliance for EcoMobility
5. List of Colorado geographic features lists
6. Japanese war crimes
7. Alan Turing
8. Element (sports)
9. Art film
10. Lists of sovereign states and dependent territories

Topic-sensitive PageRank
1. Pannier
2. Conceptual art
3. Intel
4. Global Alliance for EcoMobility
5. List of Colorado geographic features lists
6. Japanese war crimes
7. Alan Turing
